In [3]:
import mne
import os
import glob
import pandas as pd
from sklearn.feature_selection import f_regression, mutual_info_regression
import numpy as np
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from scipy.stats import binomtest
import warnings
import submitit
from imblearn.under_sampling import RandomUnderSampler


# updated classify_TF funtion 

## changes:
### 1. saving the pickle result with higher protocol adding <protocol=pickle.HIGHEST_PROTOCOL> 
### 2. put dual='auto' in selectModel
### 3. add 'new' to the name of final file 



the reason for rerunning the file was that the previous jobs apparantly  did not saved the decoding result properly so I could not load them 

In [17]:
import os
import glob
import warnings
import numpy as np
import pandas as pd
import mne
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import binomtest
import pickle

def classify_TF(eid, save_tfr=False):
    nrepeats_undersampling = 5
    n_kfolds = 5

    from sklearn.exceptions import ConvergenceWarning
    warnings.simplefilter('ignore', category=ConvergenceWarning)
    print('### WARNING: convergence warning deactivated (LinearSVC used for feature selection does not converge)')
    print(f'### PROCESSING EID: {eid}')

    eids_dir = "/mnt/data/AdaptiveControl/IBLrawdata/eid_data"
    pids_dir = "/mnt/data/AdaptiveControl/IBLrawdata/pid_data"
    probe_filelist = glob.glob(os.path.join(f'{eids_dir}/{eid}', 'probe_*.pkl'))
    pids = [x.split('probe_')[1].split('.pkl')[0] for x in probe_filelist]

    classif_df = pd.DataFrame()
    classif_output_path = os.path.join(f'{eids_dir}/{eid}', 'classification_screener_new.pkl')

    for pid in pids:
        pid_path = f'{pids_dir}/{pid}'

        # target file
        epoch_file = os.path.join(pid_path, f'lfp_{pid}_epoched_csd.fif')

        if not os.path.exists(epoch_file):
            print(f'epoch file expected at {epoch_file} not found, skip it...')
            continue

        tfr_dir = f'/mnt/data/AdaptiveControl/IBLrawdata/TF_data_large/{pid}'
        tfr_file = os.path.join(tfr_dir, 'tfr_multitaper_fast.h5')

        if not os.path.exists(tfr_dir):
            os.makedirs(tfr_dir)
            os.chmod(tfr_dir, 0o775)

        if not os.path.exists(tfr_file):
            # file based on average TFR
            epochs = mne.read_epochs(epoch_file, preload=True)

            # crude TF definition for classification
            freqs = np.concatenate([np.arange(2, 20, 2), np.arange(20, 48, 4), np.arange(60, 200, 40)])
            n_cycles = freqs / 3
            time_bandwidth = 3

            tfr_epoch = epochs.compute_tfr(
                method='multitaper',
                freqs=freqs,
                n_cycles=n_cycles,
                time_bandwidth=time_bandwidth,
                tmin=-0.3, tmax=0.3,
                decim=2,
                n_jobs=4
            )

            if save_tfr:
                tfr_epoch.save(tfr_file, overwrite=True)

        else:
            tfr_epoch = mne.time_frequency.read_tfrs(tfr_file)

        # process metadata
        constrast0_indices = np.where(((tfr_epoch.metadata['contrastRight'].isna()) & (tfr_epoch.metadata['contrastLeft'] == 0)) |
                                      (tfr_epoch.metadata['contrastLeft'].isna()) & (tfr_epoch.metadata['contrastRight'] == 0))[0]
        constrastHigh_indices = np.where(((tfr_epoch.metadata['contrastRight'].isna()) & (tfr_epoch.metadata['contrastLeft'] > 0.1)) |
                                         (tfr_epoch.metadata['contrastLeft'].isna()) & (tfr_epoch.metadata['contrastRight'] > 0.1))[0]
        constrastLow_indices = np.where(((tfr_epoch.metadata['contrastRight'].isna()) & (tfr_epoch.metadata['contrastLeft'] < 0.2)) |
                                        (tfr_epoch.metadata['contrastLeft'].isna()) & (tfr_epoch.metadata['contrastRight'] < 0.2))[0]
        constrastLeft_indices = np.where(((tfr_epoch.metadata['contrastRight'].isna()) & (tfr_epoch.metadata['contrastLeft'] > 0)))[0]
        constrastRight_indices = np.where(((tfr_epoch.metadata['contrastLeft'].isna()) & (tfr_epoch.metadata['contrastRight'] > 0)))[0]
        blockBalanced_indices = np.where((tfr_epoch.metadata['probabilityLeft'] == 0.5))[0]
        blockRight_indices = np.where((tfr_epoch.metadata['probabilityLeft'] < 0.45))[0]
        blockLeft_indices = np.where((tfr_epoch.metadata['probabilityLeft'] > 0.45))[0]

        # load tfrdata in ram
        all_tfrdata = tfr_epoch.get_data()
        all_tfrdata = np.squeeze(all_tfrdata)

        all_chnames = tfr_epoch.ch_names

        for chan, ch_name in enumerate(all_chnames):
            print(f'### PROCESSING CHANNEL: {ch_name}')

            clf = Pipeline([
                ('scaling', StandardScaler()),
                ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", max_iter=1000, dual='auto'))),
                ('classification', LogisticRegression())
            ])

            # build labels (i.e. y in the logistic regression / classification)
            labels_nostim = constrast0_indices
            labels_stim = constrastHigh_indices
            labels_nostim_stim = np.concatenate((np.zeros((labels_nostim.shape[0], 1)), np.ones((labels_stim.shape[0], 1))), axis=0)
            labels_left = constrastLeft_indices
            labels_right = constrastRight_indices
            labels_left_right = np.concatenate((np.zeros((labels_left.shape[0], 1)), np.ones((labels_right.shape[0], 1))), axis=0)
            labels_biasleft = np.setdiff1d(blockLeft_indices, constrast0_indices)
            labels_biasright = np.setdiff1d(blockRight_indices, constrast0_indices)
            labels_biasleft_biasright = np.concatenate((np.zeros((labels_biasleft.shape[0], 1)), np.ones((labels_biasright.shape[0], 1))), axis=0)

            # electrode loop
            ch_tfrdata = np.squeeze(all_tfrdata[:, chan, :, :])

            # first classification: nostim_stim
            X = np.concatenate((ch_tfrdata[labels_nostim, :, :], ch_tfrdata[labels_stim, :, :]), axis=0)
            X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])  # flatten time-frequency axes
            y = np.ravel(labels_nostim_stim)
            test_counts = 0
            success_counts = 0
            keep_counts = []
            keep_successes = []

            for rus_idx in range(nrepeats_undersampling):
                rus = RandomUnderSampler(random_state=42)
                X_res, y_res = rus.fit_resample(X, y)
                skf = StratifiedKFold(n_splits=n_kfolds)
                skf.get_n_splits(X_res, y_res)

                for i, (train_index, test_index) in enumerate(skf.split(X_res, y_res)):
                    clf.fit(X_res[train_index, :], y_res[train_index])
                    test_counts += test_index.shape[0]
                    success_counts += int(clf.score(X_res[test_index, :], y_res[test_index]) * test_index.shape[0])
                keep_counts.append(test_counts)
                keep_successes.append(success_counts)

            cv_acc_nostim_stim = np.nanmean(success_counts) / np.nanmean(test_counts)
            stats_binom = binomtest(k=int(np.nanmean(success_counts)), n=int(np.nanmean(test_counts)), p=0.5, alternative="greater")
            cv_p_nostim_stim = stats_binom.pvalue
            print(f'Global nostim_stim accuracy: {i}: {cv_acc_nostim_stim} / pvalue={cv_p_nostim_stim}')

            # second classification: left_right
            X = np.concatenate((ch_tfrdata[labels_left, :, :], ch_tfrdata[labels_right, :, :]), axis=0)
            X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])  # flatten time-frequency axes
            y = np.ravel(labels_left_right)
            test_counts = 0
            success_counts = 0
            keep_counts = []
            keep_successes = []

            for rus_idx in range(nrepeats_undersampling):
                rus = RandomUnderSampler(random_state=42)
                X_res, y_res = rus.fit_resample(X, y)
                skf = StratifiedKFold(n_splits=n_kfolds)
                skf.get_n_splits(X_res, y_res)

                for i, (train_index, test_index) in enumerate(skf.split(X_res, y_res)):
                    clf.fit(X_res[train_index, :], y_res[train_index])
                    test_counts += test_index.shape[0]
                    success_counts += int(clf.score(X_res[test_index, :], y_res[test_index]) * test_index.shape[0])
                keep_counts.append(test_counts)
                keep_successes.append(success_counts)

            cv_acc_left_right = np.nanmean(success_counts) / np.nanmean(test_counts)
            stats_binom = binomtest(k=int(np.nanmean(success_counts)), n=int(np.nanmean(test_counts)), p=0.5, alternative="greater")
            cv_p_left_right = stats_binom.pvalue
            print(f'Global left_right accuracy: {i}: {cv_acc_left_right} / pvalue={cv_p_left_right}')

            # third classification: biasleft_biasright
            X = np.concatenate((ch_tfrdata[labels_biasleft, :, :], ch_tfrdata[labels_biasright, :, :]), axis=0)
            X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])  # flatten time-frequency axes
            y = np.ravel(labels_biasleft_biasright)
            test_counts = 0
            success_counts = 0
            keep_counts = []
            keep_successes = []

            for rus_idx in range(nrepeats_undersampling):
                rus = RandomUnderSampler(random_state=42)
                X_res, y_res = rus.fit_resample(X, y)
                skf = StratifiedKFold(n_splits=n_kfolds)
                skf.get_n_splits(X_res, y_res)

                for i, (train_index, test_index) in enumerate(skf.split(X_res, y_res)):
                    clf.fit(X_res[train_index, :], y_res[train_index])
                    test_counts += test_index.shape[0]
                    success_counts += int(clf.score(X_res[test_index, :], y_res[test_index]) * test_index.shape[0])
                keep_counts.append(test_counts)
                keep_successes.append(success_counts)

            cv_acc_biasleft_biasright = np.nanmean(success_counts) / np.nanmean(test_counts)
            stats_binom = binomtest(k=int(np.nanmean(success_counts)), n=int(np.nanmean(test_counts)), p=0.5, alternative="greater")
            cv_p_biasleft_biasright = stats_binom.pvalue
            print(f'Global biasleft_biasright accuracy: {i}: {cv_acc_biasleft_biasright} / pvalue={cv_p_biasleft_biasright}')

            row_df = pd.DataFrame({
                'nostim_stim_acc': cv_acc_nostim_stim,
                'nostim_stim_pval': cv_p_nostim_stim,
                'left_right_acc': cv_acc_left_right,
                'left_right_pval': cv_p_left_right,
                'biasleft_biasright_acc': cv_acc_biasleft_biasright,
                'biasleft_biasright_pval': cv_p_biasleft_biasright
            }, index=[chan])
            row_df['pid'] = pid
            row_df['ch_name'] = ch_name
            row_df['ch_idx'] = chan
            classif_df = pd.concat([classif_df, row_df], axis=0)

    try:
        with open(classif_output_path, 'wb') as f:
            pickle.dump(classif_df, f, protocol=pickle.HIGHEST_PROTOCOL)
        print(f'Successfully saved classification data to {classif_output_path}')
    except Exception as e:
        print(f'Error saving classification data: {e}')

    return classif_df


# submit the funciton

In [18]:
base_dir='/mnt/data/AdaptiveControl/IBLrawdata/'
eids_dir = base_dir + '/eid_data'
pids_dir = base_dir + '/pid_data'

# list all eids of interest
eids=os.listdir(eids_dir)

local_computing=False

if local_computing:
  
  for eid in eids:
    
    classif_df=classify_TF(eid)

else:

  # prepare executor
  executor = submitit.AutoExecutor(folder="classif_logs")

  # define maxjobs to a low value to illustrate
  maxjobs=10000

  # pass parameter to the executor
  executor.update_parameters(slurm_array_parallelism=maxjobs, mem_gb=16, timeout_min=600, slurm_partition="CPU", cpus_per_task=5)

  # execute the job (note the .map_array command that different from the .submit command used above)
  jobs = executor.map_array(classify_TF, eids)  # just a list of jobs

  print('### all jobs submitted ###')
  print('the kernel will now be killed (and your notebook will crash) but you can see that your jobs keep running by typing squeue in the terminal')
  print('crucially, the content of tuto_output/ will still be updated in the background!')

  
  

### all jobs submitted ###
the kernel will now be killed (and your notebook will crash) but you can see that your jobs keep running by typing squeue in the terminal
crucially, the content of tuto_output/ will still be updated in the background!


# test submit 

In [15]:
eids =['aa3432cd-62bd-40bc-bc1c-a12d53bcbdcf']

base_dir='/mnt/data/AdaptiveControl/IBLrawdata/'
eids_dir = base_dir + '/eid_data'
pids_dir = base_dir + '/pid_data'

print(eids)
local_computing=False

if local_computing:
  
  for eid in eids:
    
    classif_df=classify_TF(eid)

else:

  # prepare executor
  executor = submitit.AutoExecutor(folder="classif_logs")

  # define maxjobs to a low value to illustrate
  maxjobs=10000

  # pass parameter to the executor
  executor.update_parameters(slurm_array_parallelism=maxjobs, mem_gb=12, timeout_min=600, slurm_partition="CPU", cpus_per_task=4)

  # execute the job (note the .map_array command that different from the .submit command used above)
  jobs = executor.map_array(classify_TF, eids)  # just a list of jobs

  print('### all jobs submitted ###')
  print('the kernel will now be killed (and your notebook will crash) but you can see that your jobs keep running by typing squeue in the terminal')
  print('crucially, the content of tuto_output/ will still be updated in the background!')

  
  

['aa3432cd-62bd-40bc-bc1c-a12d53bcbdcf']
### all jobs submitted ###
the kernel will now be killed (and your notebook will crash) but you can see that your jobs keep running by typing squeue in the terminal
crucially, the content of tuto_output/ will still be updated in the background!


# original function 

In [ ]:
def classify_TF(eid, save_tfr=False):
  
  nrepeats_undersampling=5
  n_kfolds=5
  
  from sklearn.exceptions import ConvergenceWarning
  warnings.simplefilter('ignore', category=ConvergenceWarning)
  print('### WARNING: convergence warning deactivated (LinearSVC used for feature selection does not converge)')
  print(f'### PROCESSING EID: {eid}')
  
  probe_filelist=glob.glob(os.path.join(f'{eids_dir}/{eid}','probe_*.pkl'))
  pids=[x.split('probe_')[1].split('.pkl')[0] for x in probe_filelist]
  
  classif_df=pd.DataFrame()
  classif_output_path=os.path.join(os.path.join(f'{eids_dir}/{eid}','classification_screener_new.pkl'))

  for pid in pids:
    
    pid_path = f'{pids_dir}/{pid}'
    
    # target file
    epoch_file = os.path.join(pid_path, f'lfp_{pid}_epoched_csd.fif')
    
    if not os.path.exists(epoch_file):
      print(f'epoch file expected at {epoch_file} not found, skip it...')
      continue

    tfr_dir=f'/mnt/data/AdaptiveControl/IBLrawdata/TF_data_large/{pid}'
    tfr_file = os.path.join(tfr_dir,'tfr_multitaper_fast.h5')

    if not os.path.exists(tfr_dir):
      os.makedirs(tfr_dir)
      os.chmod(tfr_dir, 0o775)
    
    if not os.path.exists(tfr_file):
      
      # file based on average TFR
      epochs = mne.read_epochs(epoch_file, preload=True)

      # crude TF definition for classification
      freqs = np.concatenate([np.arange(2, 20, 2), np.arange(20, 48, 4), np.arange(60, 200, 40)])
      n_cycles = freqs / 3
      time_bandwidth = 3

      tfr_epoch=epochs.compute_tfr(
              method= 'multitaper',
              freqs= freqs,
              n_cycles = n_cycles,
              time_bandwidth=time_bandwidth,
              tmin=-0.3, tmax=0.3,
              decim=2,
              n_jobs=4
      )
      
      if save_tfr:
        tfr_epoch.save(tfr_path, overwrite=True)

    else:
      
      tfr_epoch=mne.time_frequency.read_tfrs(tfr_file)

    ### process metadata
    #
    constrast0_indices=np.where(((tfr_epoch.metadata['contrastRight'].isna()) & (tfr_epoch.metadata['contrastLeft']==0)) |
    (tfr_epoch.metadata['contrastLeft'].isna()) & (tfr_epoch.metadata['contrastRight']==0))[0]
    #
    constrastHigh_indices=np.where(((tfr_epoch.metadata['contrastRight'].isna()) & (tfr_epoch.metadata['contrastLeft']>0.1)) |
    (tfr_epoch.metadata['contrastLeft'].isna()) & (tfr_epoch.metadata['contrastRight']>0.1))[0]
    #
    constrastLow_indices=np.where(((tfr_epoch.metadata['contrastRight'].isna()) & (tfr_epoch.metadata['contrastLeft']<0.2)) |
    (tfr_epoch.metadata['contrastLeft'].isna()) & (tfr_epoch.metadata['contrastRight']<0.2))[0]
    #
    constrastLeft_indices=np.where(((tfr_epoch.metadata['contrastRight'].isna()) & (tfr_epoch.metadata['contrastLeft']>0)))[0]
    constrastRight_indices=np.where(((tfr_epoch.metadata['contrastLeft'].isna()) & (tfr_epoch.metadata['contrastRight']>0)))[0]
    #
    blockBalanced_indices=np.where((tfr_epoch.metadata['probabilityLeft']==0.5))[0]
    blockRight_indices=np.where((tfr_epoch.metadata['probabilityLeft']<0.45))[0]
    blockLeft_indices=np.where((tfr_epoch.metadata['probabilityLeft']>0.45))[0]

  
    # load tfrdata in ram
    all_tfrdata=tfr_epoch.get_data()
    all_tfrdata=np.squeeze(all_tfrdata)
    
    all_chnames= tfr_epoch.ch_names
        
    for chan, ch_name in enumerate(all_chnames):
      print(f'### PROCESSING CHANNEL: {ch_name}')
      
      clf = Pipeline([
        ('scaling', StandardScaler()),
        ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", max_iter=1000))),
        ('classification', LogisticRegression())
      ])

      # build labels (i.e. y in the logistic regression / classification)
      labels_nostim=constrast0_indices
      labels_stim=constrastHigh_indices
      labels_nostim_stim = np.concatenate((np.zeros((labels_nostim.shape[0],1)), np.ones((labels_stim.shape[0],1))),axis=0)
      #
      labels_left=constrastLeft_indices
      labels_right=constrastRight_indices
      labels_left_right = np.concatenate((np.zeros((labels_left.shape[0],1)), np.ones((labels_right.shape[0],1))),axis=0)
      #
      labels_biasleft=np.setdiff1d(blockLeft_indices,constrast0_indices)
      labels_biasright=np.setdiff1d(blockRight_indices,constrast0_indices)
      labels_biasleft_biasright = np.concatenate((np.zeros((labels_biasleft.shape[0],1)), np.ones((labels_biasright.shape[0],1))),axis=0)

      #### electrode loop
      # 
      ch_tfrdata=np.squeeze(all_tfrdata[:,chan,:,:])

      #### first classification: nostim_stim
      # build X and y
      X=np.concatenate((ch_tfrdata[labels_nostim,:,:],ch_tfrdata[labels_stim,:,:]),axis=0)
      X=X.reshape(X.shape[0], X.shape[1]*X.shape[2]) # flatten time-frequency axes
      # format labels
      y=np.ravel(labels_nostim_stim)
      # classify nostim_stim 
      test_counts=0
      success_counts=0
      keep_counts=[]
      keep_successes=[]
      for rus_idx in range(nrepeats_undersampling):
        
        rus = RandomUnderSampler(random_state=42)
        X_res, y_res = rus.fit_resample(X, y)
        skf = StratifiedKFold(n_splits=n_kfolds)
        skf.get_n_splits(X_res, y_res)
        
        for i, (train_index, test_index) in enumerate(skf.split(X_res, y_res)):
          # fit
          clf.fit(X_res[train_index,:], y_res[train_index])
          test_counts=test_counts+test_index.shape[0]
          success_counts=success_counts+int(clf.score(X_res[test_index,:],y_res[test_index])*test_index.shape[0])
        #
        keep_counts.append(test_counts)
        keep_successes.append(success_counts)
      
      cv_acc_nostim_stim=np.nanmean(success_counts)/np.nanmean(test_counts)
      stats_binom=binomtest(k=int(np.nanmean(success_counts)), n=int(np.nanmean(test_counts)), p=0.5, alternative="greater")
      cv_p_nostim_stim=stats_binom.pvalue
      print(f'Global nostim_stim accuracy: {i}: {cv_acc_nostim_stim} / pvalue={cv_p_nostim_stim}')

      #### second classification: left_right
      # build X and y
      X=np.concatenate((ch_tfrdata[labels_left,:,:],ch_tfrdata[labels_right,:,:]),axis=0)
      X=X.reshape(X.shape[0], X.shape[1]*X.shape[2]) # flatten time-frequency axes
      # format labels
      y=np.ravel(labels_left_right)
      # classify nostim_stim 
      test_counts=0
      success_counts=0
      keep_counts=[]
      keep_successes=[]
      for rus_idx in range(nrepeats_undersampling):
        
        rus = RandomUnderSampler(random_state=42)
        X_res, y_res = rus.fit_resample(X, y)
        skf = StratifiedKFold(n_splits=n_kfolds)
        skf.get_n_splits(X_res, y_res)
        
        for i, (train_index, test_index) in enumerate(skf.split(X_res, y_res)):
          # fit
          clf.fit(X_res[train_index,:], y_res[train_index])
          test_counts=test_counts+test_index.shape[0]
          success_counts=success_counts+int(clf.score(X_res[test_index,:],y_res[test_index])*test_index.shape[0])
        #
        keep_counts.append(test_counts)
        keep_successes.append(success_counts)
        
      cv_acc_left_right=np.nanmean(success_counts)/np.nanmean(test_counts)
      stats_binom=binomtest(k=int(np.nanmean(success_counts)), n=int(np.nanmean(test_counts)), p=0.5, alternative="greater")
      cv_p_left_right=stats_binom.pvalue
      print(f'Global left_right accuracy: {i}: {cv_acc_left_right} / pvalue={cv_p_left_right}')
        
      #### third classification: biasleft_biasright
      # build X and y
      X=np.concatenate((ch_tfrdata[labels_biasleft,:,:],ch_tfrdata[labels_biasright,:,:]),axis=0)
      X=X.reshape(X.shape[0], X.shape[1]*X.shape[2]) # flatten time-frequency axes
      # format labels
      y=np.ravel(labels_biasleft_biasright)
      # classify nostim_stim 
      test_counts=0
      success_counts=0
      keep_counts=[]
      keep_successes=[]
      for rus_idx in range(nrepeats_undersampling):
        
        rus = RandomUnderSampler(random_state=42)
        X_res, y_res = rus.fit_resample(X, y)
        skf = StratifiedKFold(n_splits=n_kfolds)
        skf.get_n_splits(X_res, y_res)
        
        for i, (train_index, test_index) in enumerate(skf.split(X_res, y_res)):
          # fit
          clf.fit(X_res[train_index,:], y_res[train_index])
          test_counts=test_counts+test_index.shape[0]
          success_counts=success_counts+int(clf.score(X_res[test_index,:],y_res[test_index])*test_index.shape[0])
        #
        keep_counts.append(test_counts)
        keep_successes.append(success_counts)
      
      cv_acc_biasleft_biasright=np.nanmean(success_counts)/np.nanmean(test_counts)
      stats_binom=binomtest(k=int(np.nanmean(success_counts)), n=int(np.nanmean(test_counts)), p=0.5, alternative="greater")
      cv_p_biasleft_biasright=stats_binom.pvalue
      print(f'Global biasleft_biasright accuracy: {i}: {cv_acc_biasleft_biasright} / pvalue={cv_p_biasleft_biasright}')
      
      row_df=pd.DataFrame({
        'nostim_stim_acc': cv_acc_nostim_stim,
        'nostim_stim_pval': cv_p_nostim_stim,
        'left_right_acc': cv_acc_left_right,
        'left_right_pval': cv_p_left_right,
        'biasleft_biasright_acc': cv_acc_biasleft_biasright, 
        'biasleft_biasright_pval': cv_p_biasleft_biasright}, index=[chan])
      row_df['pid']=pid
      row_df['ch_name']=ch_name
      row_df['ch_idx']=chan
      classif_df=pd.concat([classif_df,row_df],axis=0)
      
  classif_df.to_pickle(classif_output_path)
  
  return  classif_df         
  
      
    
    
